In [848]:
from tkinter import *
import pandas as pd
import nltk

In [849]:
def CrearVentana():
    nuevoRoot = Tk()
    nuevoRoot.title("Grupo X - Busqueda de noticias") # Escribir nombre de la ventana
    nuevoRoot.resizable(False, False) # Si se puede escalar a lo ancho y a lo alto respectivamente
    nuevoRoot.iconbitmap("icono256.ico") # Escoger imagen que acompaña al titulo
    return nuevoRoot

In [850]:
def CrearFrame(nuevoColor):
    nuevoFrame = Frame(root) # Asociar a la ventana
    nuevoFrame.config(bg=nuevoColor) #Establecer un color al contenedor   
    # Esta linea siempre tiene que ir al final de la funcion:
    nuevoFrame.pack() # Reescalar el frame cuando se reescale la ventana
    return nuevoFrame

In [851]:
def VaciarContenedorNoticia():
    contenedorNoticia = Text(frameResultados, width=50, height=14, padx=10) # Crear el cuadro de texto de varias filas
    contenedorNoticia.insert(END, "")
    scrollNoticia = Scrollbar(frameResultados, command=contenedorNoticia.yview) # Crear y escalar el scroll, y asociarlo a su cuadro de texto
    scrollNoticia.grid(row=1, column=3, rowspan=2, padx=(0, 20), pady=(0, 20), sticky="nsw") # Ubicar el scroll
    contenedorNoticia.grid(row=1, column=2, rowspan=2, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
    contenedorNoticia.config(yscrollcommand=scrollNoticia.set) # Para que el scroll muestre a qué altura se encuentra el cuadro de texto asociado

In [852]:
def MostrarNoticia():
    if(valorMostradoRanking.get() != "No hay noticias"):
        f = open(valorMostradoRanking.get(), "r")
        leer = f.read()
        contenedorNoticia = Text(frameResultados, width=50, height=14, padx=10) # Crear el cuadro de texto de varias filas
        contenedorNoticia.insert(END, leer)
        scrollNoticia = Scrollbar(frameResultados, command=contenedorNoticia.yview) # Crear y escalar el scroll, y asociarlo a su cuadro de texto
        scrollNoticia.grid(row=1, column=3, rowspan=2, padx=(0, 20), pady=(0, 20), sticky="nsw") # Ubicar el scroll
        contenedorNoticia.grid(row=1, column=2, rowspan=2, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
        contenedorNoticia.config(yscrollcommand=scrollNoticia.set) # Para que el scroll muestre a qué altura se encuentra el cuadro de texto asociado
        f.close()

In [853]:
def BuscarNoticias():
    VaciarContenedorNoticia()
    # Buscar las noticias que cumplan los parametros
    if(cuadroConsulta.get()!=""): # La formulas pueden dar errores si la consulta tiene 0 palabras
        rutasEegidas = list()
        df = pd.read_csv("rutas.csv")
        try:
            if(valorMostradoFiltrar.get()=="Todos"):
                for i in range(len(df)):
                    rutasEegidas.append(df["rutas"][i])
            elif(valorMostradoFiltrar.get()=="El Mundo"):    
                for i in range(len(df)):
                    if(df["Noticiario"][i]=="ElMundo"):
                        rutasEegidas.append(df["rutas"][i])
            elif(valorMostradoFiltrar.get()=="El Pais"):    
                for i in range(len(df)):
                    if(df["Noticiario"][i]=="ElPais"):
                        rutasEegidas.append(df["rutas"][i])
            elif(valorMostradoFiltrar.get()=="20 minutos"):    
                for i in range(len(df)):
                    if(df["Noticiario"][i]=="20minutos"):
                        rutasEegidas.append(df["rutas"][i])
            # Aqui falta ordenar las noticias por similitud a la consulta
            rutasEegidas = rutasEegidas[0:int(valorMostradoN.get())]
            valorMostradoRanking.set(rutasEegidas[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
            desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *rutasEegidas) # Crear el desplegable
            desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable
        except:
            rutasEegidas = {"No hay noticias"}
            valorMostradoRanking.set("No hay noticias") # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
            desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *rutasEegidas) # Crear el desplegable
            desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable

In [854]:
root = CrearVentana()  # Crear frame de la ventana
frameConsulta = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)
frameAjustes = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)
frameResultados = CrearFrame("#EEEEEE") # Crear contenedor para un grupo de elementos (como un div de HTML)

labelConsulta = Label(frameConsulta, text="Consulta:").grid(row=0, column=0, padx=10, pady=20, sticky="e") # Crear y ubicar el texto
cuadroConsulta = Entry(frameConsulta)
cuadroConsulta.grid(row=0, column=1, padx=10, pady=20, sticky="ew") # Crear y ubicar el cuadro de texto

labelTopN = Label(frameAjustes, text="TOP-N:").grid(row=0, column=0, padx=(20, 0), sticky="e") # Crear y ubicar el texto
opcionesN = [2, 3, 4, 5, 6, 7, 8, 9, 30] # Opciones que se mostrarán en el desplegable de TOP-N
valorMostradoN = StringVar(frameAjustes) # Para recoger el dato seleccionado
valorMostradoN.set(opcionesN[3]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableN = OptionMenu(frameAjustes, valorMostradoN, *opcionesN) # Crear el desplegable
desplegableN.grid(row=0, column=1, padx=(0, 20), sticky="w") # Ubicar el desplegable

labelFiltrar = Label(frameAjustes, text="Filtrar:").grid(row=0, column=2, padx=(20, 0), sticky="e") # Crear y ubicar el texto
opcionesFiltrar = ["Todos", "El Mundo", "El Pais", "20 minutos"] # Opciones que se mostrarán en el desplegable de Filtrar
valorMostradoFiltrar = StringVar(frameAjustes) # Para recoger el dato seleccionado
valorMostradoFiltrar.set(opcionesFiltrar[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableFiltrar = OptionMenu(frameAjustes, valorMostradoFiltrar, *opcionesFiltrar) # Crear el desplegable
desplegableFiltrar.grid(row=0, column=3, padx=(0, 20), sticky="w") # Ubicar el desplegable

botonBuscar = Button(frameAjustes, text="Buscar", command=BuscarNoticias).grid(row=0, column=4, padx=20, pady=10) # Crear y ubicar el botón

labelRanking = Label(frameResultados, text="Ranking:").grid(row=1, column=0, padx=(20, 0), pady=(20, 10), sticky="se") # Crear y ubicar el texto
opcionesRanking = ["No hay noticias"] # Opciones que se mostrarán en el desplegable del ranking
valorMostradoRanking = StringVar(frameResultados) # Para recoger el dato seleccionado
valorMostradoRanking.set(opcionesRanking[0]) # Por defecto se mostrará y se escogerá la opción indicada en el enunciado
desplegableRanking = OptionMenu(frameResultados, valorMostradoRanking, *opcionesRanking) # Crear el desplegable
desplegableRanking.grid(row=1, column=1, padx=(0, 20), pady=(20, 10), sticky="sw") # Ubicar el desplegable

botonLeer = Button(frameResultados, text="Leer noticia", command=MostrarNoticia).grid(row=2, column=0, columnspan=2, padx=20, pady=10, sticky="new") # Crear y ubicar el botón

labelNoticia = Label(frameResultados, text="Texto de la noticia:").grid(row=0, column=2, sticky="w", padx=20, pady=(20, 0)) # Crear y ubicar el texto
contenedorNoticia = Text(frameResultados, width=50, height=14, padx=10) # Crear el cuadro de texto de varias filas
scrollNoticia = Scrollbar(frameResultados, command=contenedorNoticia.yview) # Crear y escalar el scroll, y asociarlo a su cuadro de texto
scrollNoticia.grid(row=1, column=3, rowspan=2, padx=(0, 20), pady=(0, 20), sticky="nsw") # Ubicar el scroll
contenedorNoticia.grid(row=1, column=2, rowspan=2, padx=(20, 0), pady=(0, 20)) # Ubicar el cuadro de texto de varias filas
contenedorNoticia.config(yscrollcommand=scrollNoticia.set) # Para que el scroll muestre a qué altura se encuentra el cuadro de texto asociado

root.mainloop() # Bucle infinito para escuchar acciones del usuario (siempre tiene que ir al final)

In [874]:
#nltk.download('punkt')
def CalcularSimilitud(consulta, docum):   
    # Guardar el contenido del documento
    f = open(docum, "r")
    leer = f.read()
    f.close()
    # print("Texto original:")
    # print(leer)
    # Procesar el contenido del documento   
    leer = leer.lower()
    leer = leer.replace("\n"," ")
    leer = leer.replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u").replace("ü","u")
    leer = leer.replace(","," ").replace("."," ").replace(":", " ").replace(";"," ").replace("-"," ").replace("("," ").replace(")"," ").replace("\""," ").replace("\'"," ").replace("/"," ").replace("|"," ").replace("["," ").replace("]"," ")
    leer = leer.replace("titulo "," ").replace(" entradilla "," ").replace("¿"," ").replace("?"," ").replace("!"," ").replace("¡"," ").replace("€"," € ").replace("%"," ").replace("«"," ").replace("»"," ")
    leer = leer.replace(" el "," ").replace(" la "," ").replace(" los "," ").replace(" las "," ").replace(" lo "," ")
    leer = leer.replace(" ellos "," ").replace(" ellas "," ").replace(" ello "," ").replace(" ella "," ")
    leer = leer.replace(" en "," ").replace(" que "," ").replace(" para "," ").replace(" por "," ").replace(" al "," ").replace(" le ", " ").replace(" les "," ")
    leer = leer.replace(" a pesar "," ").replace(" a partir "," ").replace(" sin embargo "," ")
    leer = leer.replace(" de "," ").replace(" del "," ").replace(" con "," ").replace(" pero "," ").replace(" aun "," ").replace(" a "," ").replace(" tan "," ").replace(" tanto "," ")
    leer = leer.replace(" segun "," ").replace(" ya "," ").replace(" y "," ").replace(" e "," ").replace(" o "," ").replace(" si "," ").replace(" tal", " ").replace(" muy "," ").replace(" pese "," ")
    leer = leer.replace(" un "," ").replace(" uno "," ").replace(" una "," ").replace(" unos "," ").replace(" unas "," ")
    leer = leer.replace(" ademas "," ").replace(" sobre "," ").replace(" entre "," ").replace(" hasta "," ").replace(" se "," ").replace(" como "," ").replace(" pues "," ").replace(" respecto "," ").replace(" junto "," ")
    leer = leer.replace(" su "," ").replace(" sus "," ").replace(" mas "," ").replace(" tambien "," ").replace(" contra "," ").replace(" aunque "," ").replace(" sino "," ").replace(" tras "," ")
    leer = leer.replace(" este "," ").replace(" esto "," ").replace(" esta "," ").replace(" estos "," ").replace(" estas "," ").replace(" asi "," ").replace(" cada "," ").replace(" durante "," ")
    leer = leer.replace(" ese "," ").replace(" eso "," ").replace(" esa "," ").replace(" esos "," ").replace(" esas "," ")
    leer = leer.replace(" es "," ").replace(" ser "," ").replace(" son "," ").replace(" sera "," ").replace(" fuese "," ").replace(" eres "," ").replace(" sea "," ").replace(" fue ", " ").replace(" sido "," ").replace(" era "," ").replace(" fueron "," ")
    leer = leer.replace(" haber "," ").replace(" ha "," ").replace(" han "," ").replace(" habian ", " ").replace(" hay ", " ").replace(" habia "," ").replace(" hallan "," ").replace(" hemos "," ").replace(" hecho "," ").replace(" haya "," ").replace(" habiendo "," ")
    leer = leer.replace(" tener "," ").replace(" tenido "," ").replace(" tuvo ", " ").replace(" tenia "," ").replace(" tiene "," ")
    leer = leer.replace(" estar "," ").replace(" estamos "," ").replace(" esten "," ").replace(" estuvo "," ")
    leer = leer.replace(" hacer "," ").replace(" hacerlo "," ").replace(" harian "," ").replace(" he "," ").replace(" hace "," ").replace(" haga "," ").replace(" hicieron "," ").replace(" hacia "," ").replace(" hechos "," ")
    leer = leer.replace(" estan "," ").replace(" ante "," ").replace(" sin ", " ").replace(" entonces ", " ").replace(" ni "," ").replace(" porque "," ").replace(" incluso "," ")
    leer = leer.replace(" nos "," ").replace(" me "," ").replace(" mi "," ").replace(" nuestro "," ").replace(" nuestra "," ")
    leer = leer.replace(" solamente "," ").replace(" desde "," ").replace(" poco "," ").replace(" pocos "," ").replace(" luego "," ").replace(" antes "," ").replace(" despues "," ").replace(" mientras "," ")
    leer = leer.replace(" parecer "," ").replace(" parece "," ").replace(" parecen "," ").replace(" parecia "," ").replace(" parecian "," ")
    leer = leer.replace(" poder "," ").replace(" puede "," ").replace(" pueden "," ").replace(" podemos "," ").replace(" puedo "," ")
    leer = leer.replace(" deber "," ").replace(" debe "," ").replace(" debemos "," ").replace(" deben "," ").replace(" debido "," ").replace(" debidos "," ").replace(" debida "," ").replace(" debidas "," ")
    leer = leer.replace(" mucho "," ").replace(" mucha "," ").replace(" muchos "," ").replace(" muchas "," ")
    leer = leer.replace(" otro "," ").replace(" otros "," ").replace(" otra "," ").replace(" otras "," ")
    leer = leer.replace(" todo "," ").replace(" toda "," ").replace(" todos "," ").replace(" todas "," ").replace(" ninguno "," ").replace(" ninguna "," ")
    leer = leer.replace(" cualquier "," ").replace(" cualquiera "," ").replace(" cualesquiera "," ")
    leer = leer.replace(" euro "," € ").replace(" euros "," € ").replace(" etc "," ").replace(" no "," ")
    leer = leer.split()
    # Eliminar las palabras repetidas del documento
    simpl = []
    [simpl.append(x) for x in leer if x not in simpl]
    print("Documento simplificado: [", len(simpl), "palabras ]")
    print(simpl)
    
    print("\nPara realizar la recuperación de texto (búsquedas) se utilizará el modelo vectorial, pre-procesando el texto junto con una lista de parada. Se calculará la similitud mediente el ángulo del coseno de los vectores, y para determinar los pesos de los términos en los documentos y la consulta (query) se empleará TF-IDF.   ")

In [875]:
CalcularSimilitud("Menos mal que me he puesto la vacuna, si no estaria en grave peligro", "ElMundo/Tecnologia/tecnologia.2021-12-10.001.txt")

Documento simplificado: [ 476 palabras ]
['videojuegos', 'the', 'game', 'awards', '2021', 'it', 'takes', 'two', 'elegido', 'mejor', 'videojuego', 'año', 'premios', 'entregado', 'madrugada', 'viernes', 'microsoft', 'theatre', 'angeles', 'moviles', 'pokemon', 'unite', 'juego', 'android', 'accion', 'aventuras', 'desarrollado', 'hazelight', 'studios', 'publicado', 'electronci', 'arts', 'recibido', 'premio', 'categoria', 'cinco', 'galardones', 'aspiraba', 'edicion', 'fundados', 'empresario', 'geoff', 'keighley', 'seis', 'años', 'estados', 'unidos', 'donde', 'entregaron', 'total', '26', 'gran', 'protagonismo', 'fundador', 'josef', 'fares', 'encargado', 'recoger', 'aventura', 'cooperacion', 'protagonizan', 'dos', 'personajes', 'cody', 'may', 'superar', 'serie', 'pruebas', 'evitar', 'acabe', 'matrimonio', 'distincion', 'figuro', 'ganadora', 'categorias', 'familiar', 'multijugador', 'forza', 'horizon', '5', 'recibio', 'mayor', 'numero', 'reconocimientos', 'largo', 'gala', 'carreras', 'playgroun